# Mitkof Stream Initiation Research 

### Dependencies

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os 

## Directory

In [ ]:
cd MitKup

In [ ]:
cd local_layers

## Reading Files

In [ ]:
# not in local layers, in mitkup 
reach = gpd.read_file("reach_Mitkup.shp")

### Inspection of files

In [ ]:
reach.columns

### Bounding Box

In [ ]:
bbox = gpd.read_file("bbox.shp")
bbox.explore

In [ ]:
smreach = gpd.read_file("reach_bbox.shp")

In [ ]:
# Checking what the values are for a certain column. 
smreach["DOWN_ID"].unique()

In [ ]:
smreach.columns

### Reach Attribute List

[Stream measurement and documentation](https://www.netmaptools.org/Pages/NetMapHelp/netmap_synthetic_stream_layer_derivation.htm?mw=MjQw&st=MQ==&sct=MA==&ms=AAAAAA==)

* area_sqkm = drainage area
* elev_m = reach elevation
* src_distance = search distance....?
* out_dist? 
* fitelev = ? 
* strm_order = [stream order documentation](https://www.netmaptools.org/Pages/NetMapHelp/stream_order.htm) the confluence of two first order channels creates a second order stream. Two second order streams create a third order channel, and so forth. stream order is sensitive to the upward extent of the channel network or locations of channels heads (e.g., number and density of first-order channels).
* azimuth_deg = The predominant direction of the stream in the downstream direction; its orientation with respect to true north (0 or 360 degrees).
sinuosity = the measure of deviation of a channel between two points from the shortest possible path is calculated as the ratio of actual channel path length divided by shortest path length (Figure 1), calculated by dividing the length of the stream channel by the straight line distance
* MNANPRC_M = The dmean annual precipitation as reflected in the stream channels (routed from drainage wings). 
* MEANANNCMS = Mean annual flow, calculated per channel segment in cubic meters per second (CMS).
* width_m = channel width
* fp_width = [flood plane width](https://www.netmaptools.org/Pages/NetMapHelp/valley_width___5x_bankfull_depth.htm), This parameter is created specifically for the Intrinsic Potential fish habitat model. It is based on valley width calculated at 5 times bankfull depth
* VWI_Floor = Valley Width Index-IP, meaning same as fp_wdith?
* MAX_GRAD_D = Maximum Downstream Gradient
* DROPMAX = ?  diff from above?
* FlowVel = stream flow velocity is predicted using the Manning equation (at bankfull hydrualic geometry).
* D_DF_Ave = The susceptibility of channels to debris flow movement, scour and deposition, in terms of probability; an empirical model. Predictions include debris flow susceptibility across entire river networks, focusing on headwater streams, and debris flow susceptibilities in mainstem (fish bearing) channels at headwater tributary junctions (e.g., likely areas of debris flows impacts from steeper headwater channels).
* GEP_CUM Generic Erosion Potential - summed downstream
* GEP Generic erosion potential segment scale
* GEP_del Generic erosion potential delivery segment scale
* Tong_class =  The NetMap-based Tongass channel classification system, that relies on remote sensing data only (digital elevation models and GIS maps), uses process groups and channel types. [see for docs](https://www.netmaptools.org/Pages/NetMapHelp/tongass_channel_classification.htm?mw=MjQw&st=MQ==&sct=MTUzMC41&ms=AAAAAA==)

### Filter by stream order

In [ ]:
smreach1 = smreach[smreach["STRM_ORDER"] == 1]

In [ ]:
smreach1.explore()

In [ ]:
smreach.explore()

# Using the linestring Z values to check highest points
determine upstream endpoint for each head segment
Use the z-values embedded in your LineString Z (you showed LINESTRING Z (...)), if present.

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString
import numpy as np

# assume smreach is your GeoDataFrame
gdf = smreach.copy()

# 1) find IDs not referenced as a DOWN_ID -> these are segments with no upstreamers
all_ids = set(gdf['ID'].astype(str))
down_ids = set(gdf['DOWN_ID'].astype(str).dropna())
head_seg_ids = [int(i) for i in (all_ids - down_ids)]

head_segs = gdf[gdf['ID'].isin(head_seg_ids)].copy()
len(head_segs), head_segs.shape


In [ ]:
from shapely.geometry import Point

def endpoint_coords(line, which=0):
    # which=0 -> start, which=-1 -> end
    coords = list(line.coords)
    return coords[which]

def endpoint_z(line, which=0):
    coords = list(line.coords)
    # coords element format (x,y,z) if z exists
    if len(coords[which]) >= 3:
        return coords[which][2]
    else:
        return None

# For each head segment, choose the endpoint with the higher elevation (upstream)
head_points = []
for idx, row in head_segs.iterrows():
    line = row.geometry
    s_z = endpoint_z(line, 0)
    e_z = endpoint_z(line, -1)
    # Prefer geometry Z if available
    if s_z is not None and e_z is not None:
        if s_z > e_z:
            upstream_pt = Point(endpoint_coords(line, 0)[:2])  # x,y
        else:
            upstream_pt = Point(endpoint_coords(line, -1)[:2])
    else:
        # fallback: choose start point if Z missing (we'll refine with DEM below)
        upstream_pt = Point(endpoint_coords(line, 0)[:2])
    head_points.append({
        'ID': row['ID'],
        'geometry': upstream_pt,
        'STRM_ORDER': row.get('STRM_ORDER', None),

        # add any other attributes you want to carry
    })

head_pts_gdf = gpd.GeoDataFrame(head_points, crs=gdf.crs)


In [ ]:
head_pts_gdf.explore()

In [ ]:
# Plot the first geodataframe
m = smreach.explore(color='blue', name='Layer 1')

# Add the second geodataframe to the same map
head_pts_gdf.explore(m=m, color='red', name='Layer 2')